# Bark text-to-speech voice cloning.
Clone voices to create speaker history prompt files (.npz) for [bark text-to-speech](https://github.com/suno-ai/bark).
(This version of the notebook is made to work on Google Colab, make sure your runtime hardware accelerator is set to GPU)

# Google Colab: Clone the repository

In [1]:
!git clone https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer/
%cd bark-voice-cloning-HuBERT-quantizer

Cloning into 'bark-voice-cloning-HuBERT-quantizer'...
remote: Enumerating objects: 1861, done.
remote: Counting objects: 100% (226/226), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 1861 (delta 130), reused 201 (delta 114), pack-reused 1635
Receiving objects: 100% (1861/1861), 319.75 MiB | 17.74 MiB/s, done.
Resolving deltas: 100% (131/131), done.
/content/bark-voice-cloning-HuBERT-quantizer


## Install packages

In [2]:
%pip install fairseq
%pip install tensorboardX
%pip install audiolm_pytorch
%pip install bark
%pip install -r requirements.txt
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 31.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.8/283.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11288380 sha256=e7bcd3289e9283be7ee7ccc8b395de7082b29091c6c0fc82160ceb17b1040471
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=1

In [3]:
from bark.generation import load_codec_model, SAMPLE_RATE, preload_models, codec_decode, generate_coarse, generate_fine, generate_text_semantic
from bark.api import generate_audio
from transformers import BertTokenizer
from encodec.utils import convert_audio
from encodec import EncodecModel
from encodec.utils import convert_audio
import numpy as np
import torch
import torchaudio

device = 'cuda' # or 'cpu'
model = load_codec_model(use_gpu=True if device == 'cuda' else False)

Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 94.8MB/s]


## Load models

In [4]:
#From https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer
from bark_hubert_quantizer.hubert_manager import HuBERTManager
hubert_manager = HuBERTManager()
hubert_manager.make_sure_hubert_installed()
hubert_manager.make_sure_tokenizer_installed()

Downloaded HuBERT


Downloaded tokenizer


'data/models/hubert/tokenizer.pth'

In [5]:
# From https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer
# Load bark_hubert_quantizer for semantic tokens
from bark_hubert_quantizer.pre_kmeans_hubert import CustomHubert
from bark_hubert_quantizer.customtokenizer import CustomTokenizer

# Load the HuBERT model
hubert_model = CustomHubert(checkpoint_path='data/models/hubert/hubert.pt').to(device)

# Load the CustomTokenizer model
tokenizer = CustomTokenizer.load_from_checkpoint('data/models/hubert/tokenizer.pth').to(device)

## Load wav and create speaker history prompt

In [6]:
# Load and pre-process the audio waveform
audio_filepath = '/content/sound.wav' # the audio you want to clone (under 13 seconds)
wav, sr = torchaudio.load(audio_filepath)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.to(device)

In [7]:
semantic_vectors = hubert_model.forward(wav, input_sample_hz=model.sample_rate)
semantic_tokens = tokenizer.get_token(semantic_vectors)

In [8]:
# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = model.encode(wav.unsqueeze(0))
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()  # [n_q, T]

In [9]:
# move codes to cpu
codes = codes.cpu().numpy()
# move semantic tokens to cpu
semantic_tokens = semantic_tokens.cpu().numpy()

In [10]:
voice_name = 'output' # whatever you want the name of the voice to be
output_path = '/content/' + voice_name + '.npz'
history_prompt = output_path
np.savez(output_path, fine_prompt=codes, coarse_prompt=codes[:2, :], semantic_prompt=semantic_tokens)

In [11]:
# Enter your prompt and speaker here
text_prompt = "Karpagam College of Engineering is a private engineering college located in Coimbatore. It was established by the Karpagam Educational Trust. The college is affiliated to Anna University."
voice_name = "output" # use your custom voice name here if you have one

In [12]:
# download and load all models
preload_models(
    text_use_gpu=True,
    text_use_small=False,
    coarse_use_gpu=True,
    coarse_use_small=False,
    fine_use_gpu=True,
    fine_use_small=False,
    codec_use_gpu=True,
    force_reload=False,
    # path="models"
)

In [13]:
# simple generation
audio_array = generate_audio(text_prompt, history_prompt, text_temp=0.7, waveform_temp=0.7)

100%|██████████| 31/31 [00:31<00:00,  1.02s/it]


In [14]:
from IPython.display import Audio
# play audio
Audio(audio_array, rate=SAMPLE_RATE)

In [15]:
from scipy.io.wavfile import write as write_wav
from google.colab import files
# save audio
filepath = '/content/' + voice_name + '.wav' # change this to your desired output path
write_wav(filepath, SAMPLE_RATE, audio_array)
files.download(filepath)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>